# Imports

In [10]:
import configparser
import json
import requests
import subprocess
from subprocess import Popen
import pandas as pd

# Get API token

In [3]:
# Create an instance
config = configparser.ConfigParser()

# Open the file
config.read("../config.ini")

# Get pmdm token from api section
token = config.get('api', 'pmdm')


# Classify text

In [4]:
# PMDM API
url = 'https://turing.iimas.unam.mx/pmdm/api/classify'

# Authorization headers
headers = {'access-token': token}

# Text example
tweet = {'tweet': 'esto es un ejemplo'}

# Request PMDM API
response = requests.post(url, headers=headers, json=tweet)

# Returns a JSON object
response.json()

{'tweet': 'esto es un ejemplo',
 'tweet_': 'esto es un ejemplo',
 'score': 0.9953651428222656,
 'lang': 'es',
 'label': 'no_evidence',
 'value': False,
 'elapsed_time': '0.0480 segs'}

## Pre-process tweets

```

Request body:

"model": "string",
"use_lower": "boolean",
"demojize": "boolean",
"process_urls": "boolean",
"process_mentions": "boolean",
"process_hashtags": "boolean",
"process_emojis": "boolean",
"process_smileys": "boolean",
"process_numbers": "boolean",
"process_escaped_chars": "boolean",
"tweet": "string"

```

In [5]:
# Portuguese example 
# 'model': 'pt'
tweet = {
    'model': 'pt',
    'use_lower': 'false',
    'demojize': 'true',
    'process_urls': 'true',
    'process_mentions': 'true',
    'process_hashtags': 'true',
    'process_emojis': 'false',
    'process_smileys': 'false',
    'process_numbers': 'false',
    'process_escaped_chars': 'false',
    'tweet': '@user1 @user2 @user3 Olha a feminazi raivosa... 🤣'
    }

response = requests.post(url, headers=headers, json=tweet)

response.json()

{'tweet': '@user1 @user2 @user3 Olha a feminazi raivosa... 🤣',
 'tweet_': '$MENTION$ $MENTION$ $MENTION$ Olha a feminazi raivosa... :rolling_on_the_floor_laughing:',
 'score': 0.9976631999015808,
 'lang': 'pt',
 'label': 'misogynistic',
 'value': True,
 'elapsed_time': '0.1337 segs'}

In [6]:
# Spanish example
# 'model': 'es'
tweet = {
  'tweet': 'jajajaja #ridicula no te vota nadie gorda sucia 🤮 https://t.co/example',
  'use_lower': 'false',
  'demojize': 'true',
  'process_urls': 'true',
  'process_mentions': 'true',
  'process_hashtags': 'true',
  'process_emojis': 'false',
  'process_smileys': 'false',
  'process_numbers': 'false',
  'process_escaped_chars': 'false',
  'model': 'es'
    }

response = requests.post(url, headers=headers, json=tweet)

response.json()

{'tweet': 'jajajaja #ridicula no te vota nadie gorda sucia 🤮 https://t.co/example',
 'tweet_': 'jajajaja $HASHTAG$ no te vota nadie gorda sucia :face_vomiting: $URL$',
 'score': 0.9990070462226868,
 'lang': 'es',
 'label': 'misogynistic',
 'value': True,
 'elapsed_time': '0.1002 segs'}

# Download file

In [21]:
# Gist file
url_gist = 'https://gist.githubusercontent.com/fer-aguirre/73635035b91c8cf0a45e66d9d51b80d8/raw/pmdmExample.csv'

# Saving location
location = './pmdmExample.csv'

# Wget command to download gist file
args = ['wget', '-O', location, url_gist]

# Execute command arguments
output = Popen(args, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

# Classify file

In [22]:
# PMDM API
url_api = 'https://turing.iimas.unam.mx/pmdm/api/classify_file'

# Token access
headers = {'access-token': token}

# File with Spanish examples
files = {'uploaded_file': open('./pmdmExample.csv', 'rb')}

# Tweet arguments required
data = {
        'model': 'es',
        'use_lower': 'false', 
        'demojize': 'true', 
        'process_urls':'true', 
        'process_mentions': 'true', 
        'process_hashtags': 'true', 
        'process_emojis': 'false', 
        'process_smileys': 'false', 
        'process_numbers': 'false', 
        'process_escaped_chars': 'false'}

# Post request
response_file = requests.post(url_api, headers=headers, files=files, data=data)

# Response request
response_file.json()

{'results': [{'tweet': '"@user1 @user2 Claro que no, que pregunta tan pendeja.🤣🤣🤣",',
   'tweet_': '"$MENTION$ $MENTION$ Claro que no, que pregunta tan pendeja.:rolling_on_the_floor_laughing::rolling_on_the_floor_laughing::rolling_on_the_floor_laughing:",',
   'lang': 'es',
   'score': 0.9993476867675781,
   'label': 'no_evidence',
   'value': False},
  {'tweet': '"@user1 Esta burra no sabe ni leer",',
   'tweet_': '"$MENTION$ Esta burra no sabe ni leer",',
   'lang': 'es',
   'score': 0.9989246726036072,
   'label': 'misogynistic',
   'value': True},
  {'tweet': '"Asquerosa y lamentable nuestra representación en el extranjero",',
   'tweet_': '"Asquerosa y lamentable nuestra representación en el extranjero",',
   'lang': 'es',
   'score': 0.9983320236206055,
   'label': 'no_evidence',
   'value': False},
  {'tweet': '"Se está haciendo de la vista gorda",',
   'tweet_': '"Se está haciendo de la vista gorda",',
   'lang': 'es',
   'score': 0.998530387878418,
   'label': 'no_evidence',
 

# Process JSON to DataFrame

In [16]:
# Get text from response
text_data = response_file.text

# Convert text to dictionary
json_dict= json.loads(text_data)

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(json_dict['results'])

# Show dataframe
df

,tweet,tweet_,lang,score,label,value
0,"""@user1 @user2 Claro que no, que pregunta tan ...","""$MENTION$ $MENTION$ Claro que no, que pregunt...",es,0.999348,no_evidence,False
1,"""@user1 Esta burra no sabe ni leer"",","""$MENTION$ Esta burra no sabe ni leer"",",es,0.998925,misogynistic,True
2,"""Asquerosa y lamentable nuestra representación...","""Asquerosa y lamentable nuestra representación...",es,0.998332,no_evidence,False
3,"""Se está haciendo de la vista gorda"",","""Se está haciendo de la vista gorda"",",es,0.998530,no_evidence,False
4,"""@user1 a vos te falta coger amargada 😜""","""$MENTION$ a vos te falta coger amargada :wink...",es,0.998933,misogynistic,True
